In [10]:
%load_ext autoreload
%autoreload 2

from cap.dehaze import dehaze as cap_dehaze
from dcp.dehaze import dehaze as dcp_dehaze
from config import config
from measure import compute_psnr, compute_ssim
import os
from glob import glob
from matplotlib import pyplot as plt
import cv2
import numpy as np
# from fastprogress.fastprogress import master_bar, progress_bar
import multiprocessing as mp
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
_RESIDE_PATH = 'D:\\data\\reside\\indoor_train'
_RESULT_PATH = '../result'

In [3]:
hazy_fold_path = os.path.join(_RESIDE_PATH,'hazy')
clear_fold_path = os.path.join(_RESIDE_PATH,'clear')

In [4]:
def get_clear(hazy_img_path, clear_fold_path):
    hazy_img_path = hazy_img_path.replace('\\', '/')
    clear_file = hazy_img_path.split('/')[-1].split('_')[0] + '.png'
    clear_img_path = os.path.join(clear_fold_path, clear_file)
    return clear_img_path

In [5]:
list_hazy_imgs = glob(os.path.join(hazy_fold_path, '*.png'))

In [6]:
list_omegas = list(np.arange(0.1, 1.0, 0.05))

In [7]:
def pipeline_finetune(hazy):
    clear = get_clear(hazy, clear_fold_path)
    clear_img = cv2.imread(clear)
    dehaze_img = dcp_dehaze(hazy, config['dcp'])
    psnr = compute_psnr(clear_img, dehaze_img)
    ssim = compute_ssim(clear_img, dehaze_img)
    return psnr, ssim

In [8]:
pool = mp.Pool(4)

In [ ]:
list_psnr = []
list_ssim = []
for omg in list_omegas:
    config['dcp']['omega'] = omg
    psnrs = []
    ssims = []
    for (psnr, ssim) in tqdm(pool.imap_unordered(pipeline_finetune,list_hazy_imgs), total = len(list_hazy_imgs)):
        list_psnr.append(np.mean(psnr))
        list_ssim.append(np.mean(ssim))

  0%|                                                                    | 0/13990 [00:00<?, ?it/s]